# Retrieving and cleaning Data

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

## Mexico DATA

### Population by State

In [146]:
import os
print(os.getcwd())
df = pd.read_csv('cpv_00.csv',usecols = ['cve_entidad','desc_entidad','cve_municipio','desc_municipio','id_indicador','2015'],skiprows=4)


/resources/labs/DS0701EN


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [147]:
df.head()

,cve_entidad,desc_entidad,cve_municipio,desc_municipio,id_indicador,2015
0,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000001,119938473
1,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000002,NaN
2,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000003,NaN
3,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000004,25.6759940264076
4,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000005,12.6075755584004


In [148]:
#[lambda x: x['id_indicador']= '1002000001']                         
df.shape

(213102, 6)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213102 entries, 0 to 213101
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   cve_entidad     213102 non-null  int64 
 1   desc_entidad    213102 non-null  object
 2   cve_municipio   213102 non-null  int64 
 3   desc_municipio  213102 non-null  object
 4   id_indicador    213102 non-null  int64 
 5   2015            147229 non-null  object
dtypes: int64(3), object(3)
memory usage: 9.8+ MB


In [150]:
df.drop(df[df.id_indicador!=1002000001].index,inplace=True)

In [151]:
df.drop(df[df.desc_municipio != 'Estatal'].index,inplace=True)
df.drop(columns=['desc_municipio'],inplace=True)
df.drop(columns=['id_indicador'],inplace=True)


In [163]:
df.rename(columns={'2015':'Population'},inplace=True)

In [164]:
df.head()

,cve_entidad,desc_entidad,cve_municipio,Population
287,1,Aguascalientes,0,1316032
1416,2,Baja California,0,3348898
2041,3,Baja California Sur,0,718384
2666,4,Campeche,0,902250
3795,5,Coahuila de Zaragoza,0,2961708


In [165]:
df.shape

(32, 4)

In [166]:
df['Population'] = df['Population'].astype(float)

In [167]:
#check for null values
df.isnull().values.any()

False

In [168]:
#verify the number of states
df.count()

cve_entidad      32
desc_entidad     32
cve_municipio    32
Population       32
dtype: int64

In [169]:
# verify consistency of data types
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 287 to 208025
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cve_entidad    32 non-null     int64  
 1   desc_entidad   32 non-null     object 
 2   cve_municipio  32 non-null     int64  
 3   Population     32 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.2+ KB


In [170]:
# Total Population in Mexico
df['Population'].sum()

119938472.0

In [171]:
#List of States
df.desc_entidad.unique

<bound method Series.unique of 287                        Aguascalientes
1416                      Baja California
2041                  Baja California Sur
2666                             Campeche
3795                 Coahuila de Zaragoza
7192                               Colima
8237                              Chiapas
18389                           Chihuahua
24157                    Ciudad de México
25706                             Durango
29187                          Guanajuato
33256                            Guerrero
40265                             Hidalgo
47526                             Jalisco
58231                              México
68936                 Michoacán de Ocampo
78633                             Morelos
81610                             Nayarit
83495                          Nuevo León
87984                              Oaxaca
136017                             Puebla
154437                          Querétaro
156154                       Quintana Roo
157

In [172]:
df['desc_entidad'].nunique()

32

In [176]:
# order the municipalities by population
df.sort_values(by=['Population'],ascending=False)

,cve_entidad,desc_entidad,cve_municipio,Population
58231,15,México,0,16225409.0
24157,9,Ciudad de México,0,8985339.0
180903,30,Veracruz de Ignacio de la Llave,0,8127832.0
47526,14,Jalisco,0,7880539.0
136017,21,Puebla,0,6183320.0
29187,11,Guanajuato,0,5864777.0
8237,7,Chiapas,0,5228711.0
83495,19,Nuevo León,0,5131938.0
68936,16,Michoacán de Ocampo,0,4599104.0
87984,20,Oaxaca,0,3976297.0


In [30]:
# saving the results
df.to_csv(r'Mexico_General_Population', index = False)

### Mexico Population by Municipality

In [96]:
df = pd.read_csv('cpv_00.csv',usecols = ['cve_entidad','desc_entidad','cve_municipio','desc_municipio','id_indicador','2010'],skiprows=4)


In [97]:
df.head()

,cve_entidad,desc_entidad,cve_municipio,desc_municipio,id_indicador,2010
0,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000001,112336538.0
1,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000002,54855231.0
2,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000003,57481307.0
3,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000004,26.8
4,0,Estados Unidos Mexicanos,0,Estados Unidos Mexicanos,1002000005,26.8


In [98]:
df.shape

(213102, 6)

In [99]:
# only include rows with total population
df.drop(df[df.id_indicador!=1002000001].index,inplace=True)
df.drop(df[df.cve_municipio == 0].index,inplace=True)
df.drop(columns=['id_indicador'],inplace=True)
df.rename(columns={'2010':'Population'},inplace=True)


In [100]:
# check for nulls values
df.isnull().values.any()

True

In [101]:
df.isnull().sum()

cve_entidad       0
desc_entidad      0
cve_municipio     0
desc_municipio    0
Population        1
dtype: int64

In [102]:
# with this we will get : a) average population by state and b) number of municipalities on each state
avgPopulation = df.groupby(['desc_entidad'])['Population'].agg(['mean', 'sum', 'count', 'std'])

In [103]:
avgPopulation 

,mean,sum,count,std
desc_entidad,,,,
Aguascalientes,107726.909091,1184996.0,11,229994.437097
Baja California,631014.000000,3155070.0,5,623641.250008
Baja California Sur,127405.200000,637026.0,5,109480.770424
Campeche,74767.363636,822441.0,11,84801.303331
Chiapas,40648.983051,4796580.0,118,64204.905396
Chihuahua,50842.761194,3406465.0,67,188789.396059
Ciudad de México,553192.500000,8851080.0,16,418758.423417
Coahuila de Zaragoza,72326.078947,2748391.0,38,154019.817197
Colima,65055.500000,650555.0,10,62251.102038


In [104]:
df[df.Population.isnull()]

,cve_entidad,desc_entidad,cve_municipio,desc_municipio,Population
18354,7,Chiapas,998,Otros municipios,NaN


In [105]:
# replacing Nan for Chiapas, Municicpality = 998, with  0
df.at[18354,'Population']= 0

In [106]:
# this should be false
df.isnull().values.any()

False

In [107]:
df.to_csv(r'Mexico_StatePopulation', index = False)

# Japan Data

## Japan Population by perfecture

In [126]:
import os
import pandas as pd
dfJapan = pd.read_csv('Data Population ALL Japan FEI_PREF_201026050834.csv',usecols = ['YEAR','AREA Code','AREA','A1101_Total population (Both sexes)[person]'],skiprows=8)



In [127]:
dfJapan.dtypes

YEAR                                            int64
AREA Code                                       int64
AREA                                           object
A1101_Total population (Both sexes)[person]    object
dtype: object

In [128]:
dfJapan['YEAR'].unique()

array([2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008,
       2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997,
       1996, 1995, 1994, 1993, 1992, 1991, 1990, 1989, 1988, 1987, 1986,
       1985, 1984, 1983, 1982, 1981, 1980, 1979, 1978, 1977, 1976, 1975])

In [129]:
dfJapan.drop(dfJapan[dfJapan.YEAR!=2018].index,inplace=True)

In [130]:
dfJapan.rename(columns={'A1101_Total population (Both sexes)[person]': 'Population'},inplace=True)

In [137]:
dfJapan['Population']= dfJapan['Population'].str.replace(",","").astype(float)


In [111]:
dfJapan.dtypes

YEAR           int64
AREA Code      int64
AREA          object
Population    object
dtype: object

In [138]:
dfJapan.head()

,YEAR,AREA Code,AREA,Population
0,2018,1000,Hokkaido,5286000.0
1,2018,2000,Aomori-ken,1263000.0
2,2018,3000,Iwate-ken,1241000.0
3,2018,4000,Miyagi-ken,2316000.0
4,2018,5000,Akita-ken,981000.0


In [141]:
# total perfectures in Japan
dfJapan['AREA'].count()

47

In [174]:
# perfectures order by Population
dfJapan.sort_values(by=['Population'],ascending=False)

,YEAR,AREA Code,AREA,Population
12,2018,13000,Tokyo-to,13822000.0
13,2018,14000,Kanagawa-ken,9177000.0
26,2018,27000,Osaka-fu,8813000.0
22,2018,23000,Aichi-ken,7537000.0
10,2018,11000,Saitama-ken,7330000.0
11,2018,12000,Chiba-ken,6255000.0
27,2018,28000,Hyogo-ken,5484000.0
0,2018,1000,Hokkaido,5286000.0
39,2018,40000,Fukuoka-ken,5107000.0
21,2018,22000,Shizuoka-ken,3659000.0


In [177]:
dfJapan.to_csv(r'Japan_General_Population', index = False)

### Tokyo population by Ward

### This information was gathered from wikipedia (see notebook : BeautifulSoap.ipynb)

In [20]:
import pandas as pd
# this file comes from notebook BeautifulSoap.ipynb 
df = pd.read_csv('Japan_WardTokyo.csv')
df.head()


,No.,Name,Kanji,Poblacion,Density(/km2),Area(km2),MajorDistricts
0,1,Chiyoda,千代田区,59441.0,5100.0,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,..."
1,2,Chūō,中央区,147620.0,14460.0,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb..."
2,3,Minato,港区,248071.0,12180.0,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong..."
3,4,Shinjuku,新宿区,339211.0,18620.0,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich..."
4,5,Bunkyō,文京区,223389.0,19790.0,11.29,"Hongō, Yayoi, Hakusan"


In [11]:
df.sort_values("Name")
df.shape

(23, 7)

In [21]:
# include Postal Codes 
PostalCode = [13101,13102,13103,13104,13105,13106,13107,13108,13109,13110,13111,13112,13113,13114,13115,13116,13117,13118,13119,13120,13121,13122,13123]
df['PostalCode'] = PostalCode

In [22]:
df.head()

,No.,Name,Kanji,Poblacion,Density(/km2),Area(km2),MajorDistricts,PostalCode
0,1,Chiyoda,千代田区,59441.0,5100.0,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,...",13101
1,2,Chūō,中央区,147620.0,14460.0,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb...",13102
2,3,Minato,港区,248071.0,12180.0,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong...",13103
3,4,Shinjuku,新宿区,339211.0,18620.0,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich...",13104
4,5,Bunkyō,文京区,223389.0,19790.0,11.29,"Hongō, Yayoi, Hakusan",13105


In [23]:
df.to_csv(r'Japan_WardTokyo_PostalCodes', index = False)

In [24]:
df = pd.read_csv('Japan_WardTokyo_PostalCodes')
df.head()

,No.,Name,Kanji,Poblacion,Density(/km2),Area(km2),MajorDistricts,PostalCode
0,1,Chiyoda,千代田区,59441.0,5100.0,11.66,"Nagatachō, Kasumigaseki, Ōtemachi, Marunouchi,...",13101
1,2,Chūō,中央区,147620.0,14460.0,10.21,"Nihonbashi, Kayabachō, Ginza, Tsukiji, Hatchōb...",13102
2,3,Minato,港区,248071.0,12180.0,20.37,"Odaiba, Shinbashi, Hamamatsuchō, Mita, Roppong...",13103
3,4,Shinjuku,新宿区,339211.0,18620.0,18.22,"Shinjuku, Takadanobaba, Ōkubo, Kagurazaka, Ich...",13104
4,5,Bunkyō,文京区,223389.0,19790.0,11.29,"Hongō, Yayoi, Hakusan",13105
